In [1]:
import pandas as pd
import re
from sklearn.feature_extraction import DictVectorizer

## Clean wines.csv
### Expand nested list of dictionaries in a df['flavor']

In [2]:
df = pd.read_csv("wines.csv")
df['flavors'] = df['flavors'].apply(lambda x: dict(zip(re.findall('\'group\W*(.*?)\W', x), re.findall('stats\W*\'count\W*(.*?)\W', x))))

In [3]:
data_dict = df['flavors'].to_list()
data_dict = [dict([a, int(x)] for a, x in b.items()) for b in data_dict]

dictvectorizer = DictVectorizer(sparse=False)
features = dictvectorizer.fit_transform(data_dict)

feature_name =dictvectorizer.get_feature_names()

df_flavor = pd.DataFrame(features, columns=feature_name)
df_flavor

,black_fruit,citrus_fruit,dried_fruit,earth,floral,microbio,non_oak,oak,red_fruit,spices,tree_fruit,tropical_fruit,vegetal
0,27.0,1859.0,36.0,983.0,79.0,1838.0,1909.0,1533.0,102.0,184.0,2161.0,130.0,506.0
1,2.0,16.0,0.0,10.0,1.0,25.0,24.0,20.0,86.0,4.0,10.0,1.0,4.0
2,32.0,1461.0,25.0,715.0,117.0,1230.0,1293.0,974.0,74.0,143.0,1741.0,109.0,343.0
3,81.0,702.0,33.0,333.0,49.0,715.0,705.0,548.0,1476.0,100.0,400.0,35.0,106.0
4,41.0,761.0,28.0,413.0,99.0,683.0,602.0,525.0,1315.0,98.0,601.0,42.0,162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,126.0,27.0,38.0,358.0,89.0,62.0,236.0,393.0,574.0,224.0,5.0,2.0,24.0
965,360.0,8.0,32.0,140.0,10.0,27.0,221.0,404.0,157.0,266.0,8.0,2.0,9.0
966,45.0,4.0,5.0,134.0,15.0,35.0,64.0,124.0,174.0,50.0,1.0,0.0,5.0
967,75.0,4.0,9.0,95.0,6.0,17.0,50.0,107.0,39.0,33.0,2.0,0.0,2.0


### Merge back to the raw datasets and normalize the data

In [4]:
#merge two dataframe
cleaned_wine = pd.merge(df,df_flavor, left_index=True, right_index=True)
cleaned_wine = cleaned_wine.drop(columns=['flavors'])

#normalize the value of flavors by dividing ratings_count
flavors = ['black_fruit', 'citrus_fruit', 'dried_fruit', 'earth', 'floral',
       'microbio', 'non_oak', 'oak', 'red_fruit', 'spices', 'tree_fruit',
       'tropical_fruit', 'vegetal']
for flavor in flavors:
    cleaned_wine[flavor] = cleaned_wine[flavor]/cleaned_wine['ratings_count']

#save the dataframe into csv file
#cleaned_wine.to_csv('cleaned_wine.csv')

cleaned_wine


,name,year,wine ID,rating,price,winery,ratings_count,region,number_of_flavors,number_of_grapes,...,earth,floral,microbio,non_oak,oak,red_fruit,spices,tree_fruit,tropical_fruit,vegetal
0,Brut Premier Champagne N.V.,N.V.,74298,4.2,65.098163,Louis Roederer,52143,Champagne Premier Cru,13,3,...,0.018852,0.001515,0.035249,0.036611,0.029400,0.001956,0.003529,0.041444,0.002493,0.009704
1,Oeil de Perdrix Brut Rosé Champagne N.V.,N.V.,1998957,3.9,80.132381,Champagne Devaux,856,Champagne,12,3,...,0.011682,0.001168,0.029206,0.028037,0.023364,0.100467,0.004673,0.011682,0.001168,0.004673
2,Grand Brut Champagne N.V.,N.V.,79160,4.2,67.333250,Perrier-Jouët,39646,Champagne,13,3,...,0.018035,0.002951,0.031025,0.032614,0.024567,0.001867,0.003607,0.043914,0.002749,0.008652
3,Brut Cuvée Champagne Rosé N.V.,N.V.,8305,4.4,69.057174,Laurent-Perrier,35543,Champagne,13,3,...,0.009369,0.001379,0.020116,0.019835,0.015418,0.041527,0.002813,0.011254,0.000985,0.002982
4,Brut Rosé Champagne N.V.,N.V.,1211816,4.4,57.029800,Billecart-Salmon,21474,Champagne,13,3,...,0.019233,0.004610,0.031806,0.028034,0.024448,0.061237,0.004564,0.027987,0.001956,0.007544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,Barbaresco 2018,2018,61144,4.1,66.194090,Prunotto,1421,Barbaresco,13,1,...,0.251935,0.062632,0.043631,0.166080,0.276566,0.403941,0.157635,0.003519,0.001407,0.016890
965,Max's Shiraz 2016,2016,4275488,3.9,85.807154,Penfolds,1093,South Australia,13,1,...,0.128088,0.009149,0.024703,0.202196,0.369625,0.143641,0.243367,0.007319,0.001830,0.008234
966,Vieilles Vignes Gevrey-Chambertin 2012,2012,1189367,4.0,105.860000,Domaine Rossignol-Trapet,326,Gevrey-Chambertin,12,1,...,0.411043,0.046012,0.107362,0.196319,0.380368,0.533742,0.153374,0.003067,0.000000,0.015337
967,Pomerol 2011,2011,1112303,4.1,108.400000,Château Vray Croix de Gay,154,Pomerol,12,6,...,0.616883,0.038961,0.110390,0.324675,0.694805,0.253247,0.214286,0.012987,0.000000,0.012987
